In [1]:
import numpy as np
import numpy.ma as ma
import scipy.stats as stat
import random

In [2]:
parameters = {}
parameters["channels"] = ["RED"]
parameters["surface functionalities"] = ["nPEG", "PEG"]
parameters["slices"] = [1, 2, 3, 4]
parameters["videos"] = [1, 2, 3, 4, 5]

channels = parameters["channels"]
surface_functionalities = parameters["surface functionalities"]
slices = parameters["slices"]
videos = parameters["videos"]

geoM2xy = {}
gSEM = {}
SM1x = {}
SM1y = {}
SM2xy = {}
npar = {}

In [ ]:
!ls 0mM/g*

In [3]:
DIR = "./3mM/"

for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            for video in videos:
                sample_name = "well{}_XY{}".format(slic, video)                
                #SM2xy[sample_name] = np.genfromtxt('SM2xy_{}.csv'.format(sample_name, delimiter=","))
                
                #npar[sample_name] = SM2xy[sample_name].shape
                geoM2xy[sample_name] = np.genfromtxt(DIR + 'geoM2xy_{}.csv'.format(sample_name, delimiter=","))
                gSEM[sample_name] = np.genfromtxt(DIR + 'gSEM_{}.csv'.format(sample_name, delimiter=","))

In [4]:
geo_slices = {}
gSEM_slices = {}
w_slices = {}
wo_slices = {}

In [5]:
#Calculate the precision weights over videos
#Alternately, can weight by the number of particles in each video
nvids = 5
nslices = 4
frames = 69

for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            video_counter = 0
            w_holder = np.zeros((nvids, frames))
            sample_name = "well{}".format(slic)
            for key in geoM2xy:
                if sample_name in key:
                    w_holder[video_counter, :] = 1/(gSEM[key]*gSEM[key])
                    video_counter = video_counter + 1
            wo_slices[sample_name] = np.sum(w_holder, axis=0)
            slice_counter = slice_counter + 1

C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in true_divide


In [6]:
#Calculate the weights SEMs and means over videos
#Remember to use alternate if not wanting to use precision weights at this level.
for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            geo_holder = np.zeros((nvids, frames))
            gSEM_holder = np.zeros((nvids, frames))
            w_holder = np.zeros((nvids, frames))
            video_counter = 0
            sample_name = "well{}".format(slic)
            for key in geoM2xy:
                if sample_name in key:
                    w_holder[video_counter, :] = (1/(gSEM[key]*gSEM[key]))/wo_slices[sample_name]
                    geo_holder[video_counter, :] = w_holder[video_counter, :] * geoM2xy[key]
                    gSEM_holder[video_counter, :] = (1/(gSEM[key]*gSEM[key]))
                    video_counter = video_counter + 1
            geo_slices[sample_name] = np.sum(geo_holder, axis=0)
            gSEM_slices[sample_name] = np.sqrt((1/np.sum(gSEM_holder, axis=0)))
            slice_counter = slice_counter + 1

C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in true_divide
C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in true_divide


In [7]:
geo = {}
gS = {}
w_slices = {}
wo_slices = {}

In [8]:
#Calculate the precision weights over slices
for channel in channels:
    counter = 0
    for surface_functionality in surface_functionalities:
        w_holder = np.zeros((nslices, frames))
        slice_counter = 0
        sample_name = "well"
        for key in geo_slices:
            if sample_name in key:
                w_holder[slice_counter, :] = 1/(gSEM_slices[key]*gSEM_slices[key])
                slice_counter = slice_counter + 1
        wo_slices[sample_name] = np.sum(w_holder, axis=0)
        counter = counter + 1

C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:10: RuntimeWarning: divide by zero encountered in true_divide


In [9]:
#Calculate the weights SEMs and means over slices
for channel in channels:
    counter = 0
    for surface_functionality in surface_functionalities:
        geo_holder = np.zeros((nslices, frames))
        gSEM_holder = np.zeros((nslices, frames))
        w_holder = np.zeros((nslices, frames))
        slice_counter = 0
        sample_name = "well"
        for key in geo_slices:
            if sample_name in key:
                w_holder[slice_counter, :] = (1/(gSEM_slices[key]*gSEM_slices[key]))/wo_slices[sample_name]
                geo_holder[slice_counter, :] = w_holder[slice_counter, :] * geo_slices[key]
                gSEM_holder[slice_counter, :] = (1/(gSEM_slices[key]*gSEM_slices[key]))
                slice_counter = slice_counter + 1
        geo[sample_name] = np.sum(geo_holder, axis=0)
        gS[sample_name] = np.sqrt((1/np.sum(gSEM_holder, axis=0)))
        counter = counter + 1

C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: invalid value encountered in true_divide
C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: divide by zero encountered in true_divide


In [10]:
np.exp(geo['well'])

array([        nan,  0.00405228,  0.00588618,  0.00635038,  0.00644694,
        0.00667509,  0.00695366,  0.00700412,  0.00708108,  0.00736413,
        0.00774171,  0.00832964,  0.00895324,  0.00962946,  0.0104997 ,
        0.01147661,  0.01297853,  0.01449897,  0.0165643 ,  0.01919505,
        0.02219748,  0.02595819,  0.03044135,  0.0363914 ,  0.04342509,
        0.05267736,  0.06341753,  0.07778845,  0.09539142,  0.11533717,
        0.13903324,  0.16927054,  0.20471697,  0.24574322,  0.2927987 ,
        0.34685478,  0.40858866,  0.47478246,  0.55123164,  0.63143628,
        0.70560046,  0.77633002,  0.83717167,  0.90485598,  0.96438056,
        0.99236305,  1.00978937,  1.00995381,  0.99995961,  0.98973298,
        0.95879981,  0.92628497,  0.88243613,  0.83617503,  0.79138841,
        0.74315757,  0.6963036 ,  0.64707582,  0.6045505 ,  0.55965563,
        0.51840289,  0.47737129,  0.45508721,  0.47435282,  0.54807941,
        0.65622025,  0.80531857,  0.96242339,  1.12942422])

In [11]:
for key in geo:
    np.savetxt(DIR+'geoM2xy_{}.csv'.format(key), geo[key], delimiter=',')
    np.savetxt(DIR+'gSEM_{}.csv'.format(key), gS[key], delimiter=',')